In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import time

with open('/content/drive/MyDrive/AdventOfCode2021/aoc16.txt', 'r') as file:
    data = [x for x in file]

In [3]:
hex_to_binary_map = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'A': '1010',
    'B': '1011',
    'C': '1100',
    'D': '1101',
    'E': '1110',
    'F': '1111'
}

def hex_to_binary(hex_string):
    binary_string = ""
    for character in hex_string:
        binary_string += hex_to_binary_map[character]
    return binary_string


In [4]:
def literal_value(packet_binary):
    groups = [packet_binary[i:i+5] for i in range(0, len(packet_binary), 5)]
    groups4 = [g[1:] for g in groups]
    binary = ''.join(groups4)
    return int(binary, 2)

In [5]:
def packet_decoder(packet):
    actual_pos = 0
    binary_data = hex_to_binary(packet[0])
    return packet_decoder_rec(actual_pos, binary_data) # 977

In [8]:
def packet_decoder_rec(pos, binary_data):
    version = int(binary_data[pos:pos+3], 2)
    type_id = int(binary_data[pos+3:pos+6], 2)
    actual_pos = pos + 6
    
    if type_id == 4:
        for i in range(actual_pos, len(binary_data), 5):
            if binary_data[i] == '0':
                literal_value(binary_data[actual_pos:i+5])
                actual_pos = i + 5
                return actual_pos, version
    else:
        length_type_id = binary_data[actual_pos]
        actual_pos += 1
        if length_type_id == '0':
            total_length_sub_packets = int(binary_data[actual_pos:actual_pos+15], 2)
            actual_pos +=15
            end = actual_pos + total_length_sub_packets

            while actual_pos < end:
                actual_pos, v_aux = packet_decoder_rec(actual_pos, binary_data)
                version += v_aux
            return actual_pos, version
        else:
            n_sub_packet = int(binary_data[actual_pos:actual_pos+11], 2)
            actual_pos += 11
            for i in range(n_sub_packet):
                actual_pos, v_aux = packet_decoder_rec(actual_pos, binary_data)
                version += v_aux
            return actual_pos, version
    return actual_pos, version

In [9]:
start_time = time.time()
print(packet_decoder(data))
print("--- %s seconds ---" % (time.time() - start_time))

(5476, 977)
--- 0.002231121063232422 seconds ---


In [10]:
def packet_decoder2(packet):
    actual_pos = 0
    binary_data = hex_to_binary(packet[0])
    return packet_decoder_rec2(actual_pos, binary_data)

In [11]:
import numpy

def packet_decoder_rec2(pos, binary_data):
    values = []
    type_id = int(binary_data[pos+3:pos+6], 2)
    actual_pos = pos + 6
    
    if type_id == 4:
        for i in range(actual_pos, len(binary_data), 5):
            if binary_data[i] == '0':
                value = literal_value(binary_data[actual_pos:i+5])
                actual_pos = i + 5
                return actual_pos, value
    else:
        length_type_id = binary_data[actual_pos]
        actual_pos += 1
        if length_type_id == '0':
            total_length_sub_packets = int(binary_data[actual_pos:actual_pos+15], 2)
            actual_pos +=15
            end = actual_pos + total_length_sub_packets

            while actual_pos < end:
                actual_pos, value = packet_decoder_rec2(actual_pos, binary_data)
                values.append(value)
        else:
            n_sub_packet = int(binary_data[actual_pos:actual_pos+11], 2)
            actual_pos += 11
            for i in range(n_sub_packet):
                actual_pos, value = packet_decoder_rec2(actual_pos, binary_data)
                values.append(value)
        if type_id == 0:
            return actual_pos, sum(values)
        elif type_id == 1:
            return actual_pos, numpy.prod(values)
        elif type_id == 2:
            return actual_pos, min(values)
        elif type_id == 3:
            return actual_pos, max(values)
        elif type_id == 5:
            val = 0
            if values[0] > values[1]:
                val = 1
            return actual_pos, val
        elif type_id == 6:
            val = 0
            if values[0] < values[1]:
                val = 1
            return actual_pos, val
        elif type_id == 7:
            val = 0
            if values[0] == values[1]:
                val = 1
            return actual_pos, val
    return actual_pos, values

In [12]:
start_time = time.time()
print(packet_decoder2(data))
print("--- %s seconds ---" % (time.time() - start_time))

(5476, 101501020883)
--- 0.008024454116821289 seconds ---
